<a href="https://colab.research.google.com/github/LxYuan0420/nlp/blob/main/notebooks/Inference_Llama_2_7B_Chat_using_AWQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Install and Load Required Libraries

In [1]:
!nvidia-smi

Sat Jan 20 03:56:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
!pip install -q -U transformers accelerate autoawq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 18.6 MB/s eta 0:00:00


In [6]:
!pip freeze | grep -E "trans|awq|acce"

accelerate==0.26.1
autoawq==0.1.8
google-cloud-translate==3.11.3
transformers==4.36.2


### 2. Load AWQ-Quantized Model from Huggingface

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "TheBloke/Llama-2-7b-Chat-AWQ"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0")
tokenizer = AutoTokenizer.from_pretrained(model_id)

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:
chat = [
   {"role": "system", "content": "You are a friendly and smart AI model that acts like a wikipedia question answering bot"},
   {"role": "user", "content": "Who is Elon Musk? Summarize your answer in point form"},
]

prompt_template = tokenizer.apply_chat_template(chat, tokenize=False)

In [10]:
prompt_template

'<s>[INST] <<SYS>>\nYou are a friendly and smart AI model that acts like a wikipedia question answering bot\n<</SYS>>\n\nWho is Elon Musk? Summarize your answer in point form [/INST]'

In [13]:
# Inference can also be done using transformers' pipeline
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

response = (pipe(prompt_template)[0]['generated_text'])

In [15]:
response = pipe(prompt_template)

In [16]:
print(response)

[{'generated_text': "<s>[INST] <<SYS>>\nYou are a friendly and smart AI model that acts like a wikipedia question answering bot\n<</SYS>>\n\nWho is Elon Musk? Summarize your answer in point form [/INST]  Of course! Elon Musk is a South African-born entrepreneur, inventor, and business magnate who has had a significant impact on various industries. Here's a summary of his life and achievements in bullet points:\n\n* Born on June 28, 1971, in Pretoria, South Africa\n* Co-founder of PayPal, an online payment system (1998)\n* Founder and CEO of SpaceX, a private space exploration company (2002)\n* Founder and CEO of Tesla, Inc., an electric vehicle manufacturer (2003)\n* Founder of Neuralink, a neurotechnology company focused on brain–machine interfaces (2016)\n* Founder of The Boring Company, a tunnel construction company (2016)\n* Co-founder of SolarCity, a solar energy services provider (2004)\n* Acquired Twitter in 2013\n* First entrepreneur to have a successful IPO with a market valua

In [17]:
print(response[0]["generated_text"])

<s>[INST] <<SYS>>
You are a friendly and smart AI model that acts like a wikipedia question answering bot
<</SYS>>

Who is Elon Musk? Summarize your answer in point form [/INST]  Of course! Elon Musk is a South African-born entrepreneur, inventor, and business magnate who has had a significant impact on various industries. Here's a summary of his life and achievements in bullet points:

* Born on June 28, 1971, in Pretoria, South Africa
* Co-founder of PayPal, an online payment system (1998)
* Founder and CEO of SpaceX, a private space exploration company (2002)
* Founder and CEO of Tesla, Inc., an electric vehicle manufacturer (2003)
* Founder of Neuralink, a neurotechnology company focused on brain–machine interfaces (2016)
* Founder of The Boring Company, a tunnel construction company (2016)
* Co-founder of SolarCity, a solar energy services provider (2004)
* Acquired Twitter in 2013
* First entrepreneur to have a successful IPO with a market valuation of over $1 billion without rai

In [18]:
model_response = response[0]["generated_text"].replace(prompt_template, "")
print(model_response)

  Of course! Elon Musk is a South African-born entrepreneur, inventor, and business magnate who has had a significant impact on various industries. Here's a summary of his life and achievements in bullet points:

* Born on June 28, 1971, in Pretoria, South Africa
* Co-founder of PayPal, an online payment system (1998)
* Founder and CEO of SpaceX, a private space exploration company (2002)
* Founder and CEO of Tesla, Inc., an electric vehicle manufacturer (2003)
* Founder of Neuralink, a neurotechnology company focused on brain–machine interfaces (2016)
* Founder of The Boring Company, a tunnel construction company (2016)
* Co-founder of SolarCity, a solar energy services provider (2004)
* Acquired Twitter in 2013
* First entrepreneur to have a successful IPO with a market valuation of over $1 billion without raising any external capital (2010)
* Ranked 21st on Forbes list of wealthiest people in the world (2020)
* Known for his innovative ideas, visionary leadership, and ambitious goal

In [19]:
chat = [
   {"role": "system", "content": "You are a friendly and smart AI model that acts like a wikipedia question answering bot"},
   {"role": "user", "content": "Who is Elon Musk? Summarize your answer in point form"},
   {"role": "assistant", "content": model_response},
   {"role": "user", "content": "Extract list of relation triplet from your above answer"},
]

continued_prompt_template = tokenizer.apply_chat_template(chat, tokenize=False)

In [21]:
continued_prompt_template

"<s>[INST] <<SYS>>\nYou are a friendly and smart AI model that acts like a wikipedia question answering bot\n<</SYS>>\n\nWho is Elon Musk? Summarize your answer in point form [/INST] Of course! Elon Musk is a South African-born entrepreneur, inventor, and business magnate who has had a significant impact on various industries. Here's a summary of his life and achievements in bullet points:\n\n* Born on June 28, 1971, in Pretoria, South Africa\n* Co-founder of PayPal, an online payment system (1998)\n* Founder and CEO of SpaceX, a private space exploration company (2002)\n* Founder and CEO of Tesla, Inc., an electric vehicle manufacturer (2003)\n* Founder of Neuralink, a neurotechnology company focused on brain–machine interfaces (2016)\n* Founder of The Boring Company, a tunnel construction company (2016)\n* Co-founder of SolarCity, a solar energy services provider (2004)\n* Acquired Twitter in 2013\n* First entrepreneur to have a successful IPO with a market valuation of over $1 billi

In [22]:
continued_response = pipe(continued_prompt_template)

In [23]:
continued_response

[{'generated_text': "<s>[INST] <<SYS>>\nYou are a friendly and smart AI model that acts like a wikipedia question answering bot\n<</SYS>>\n\nWho is Elon Musk? Summarize your answer in point form [/INST] Of course! Elon Musk is a South African-born entrepreneur, inventor, and business magnate who has had a significant impact on various industries. Here's a summary of his life and achievements in bullet points:\n\n* Born on June 28, 1971, in Pretoria, South Africa\n* Co-founder of PayPal, an online payment system (1998)\n* Founder and CEO of SpaceX, a private space exploration company (2002)\n* Founder and CEO of Tesla, Inc., an electric vehicle manufacturer (2003)\n* Founder of Neuralink, a neurotechnology company focused on brain–machine interfaces (2016)\n* Founder of The Boring Company, a tunnel construction company (2016)\n* Co-founder of SolarCity, a solar energy services provider (2004)\n* Acquired Twitter in 2013\n* First entrepreneur to have a successful IPO with a market valuat

In [26]:
print(continued_response[0]["generated_text"].replace(continued_prompt_template, ""))

  Certainly! From my previous answer, here are some relation triplets that can be extracted:
Relationship: Co-founder
* Elon Musk co-founded PayPal with Peter Thiel (1998)
Relationship: Founder and CEO
* Elon Musk founded SpaceX in 2002
* Elon Musk founded Tesla in 2003

Relationship: Acquisition
* Elon Musk acquired Twitter in 2013

Relationship: Honors and Awards
* Elon Musk received the 2007 Heinlein Prize for Advances in Space Commercialization (2007)
* Elon Musk received the 2013 National Academy of Engineering's Arthur M. Bueche Award (2013)
* Elon Musk received the 2017 Solar Energy Industry Association's (SEIA) John F. Kennedy Renewable Energy Award (2017)

Please note that these are just some of the relation triplets that can be extracted from my previous answer, and there may be more depending on how one defines and structures their data.


In [32]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!nvidia-smi

Sat Jan 20 04:14:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0              29W /  70W |   5235MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--